In [7]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


train = pd.read_csv('../../data/train.tsv', sep='\t')
test = pd.read_csv('../../data/test.tsv', sep='\t')

In [8]:
train = train[:10]
test = test[:10]

split_indexs = int(len(train)/2)
train_test = train[: split_indexs]
train_train = train[split_indexs:]

train = np.array(train)
test = np.array(test)

# train_sentiment = train[:,-1]
train = train[:, :3]

print train.shape
print test.shape

data = np.concatenate((train, test), axis=0)
print data.shape
columns = ['PhraseId', 'SentenceId', 'Phrase']
data= pd.DataFrame(data, columns=columns)
test = pd.DataFrame(test, columns=columns)
columns = ['PhraseId', 'SentenceId', 'Phrase', 'Sentiment']
train_test= pd.DataFrame(train_test, columns=columns)
train_train= pd.DataFrame(train_train, columns=columns)


(10, 3)
(10, 3)
(20, 3)


In [9]:
from textblob import TextBlob
def extract_verbs_and_adj(phrase):
    words = []
    
    tags = TextBlob(phrase).tags
    for tag in tags:
        if tag[1][:2] == "VB" or tag[1][:2] == "JJ":
            words.append(tag[0])
    return words

def looping_extract_verbs_and_adj(data_set):
    phrases_vb_adj = []

    for (index, phrase) in enumerate(data_set):
        if index % 10 == 0:
            print index
        phrases_vb_adj.append(' '.join(extract_verbs_and_adj(phrase)))
    return phrases_vb_adj

data_vb_adj = looping_extract_verbs_and_adj(data.Phrase)



0
10


In [10]:
from sklearn.feature_extraction.text import CountVectorizer

v1 = CountVectorizer(ngram_range=(0, 4),  min_df=2, max_df=0.95, max_features=1000)
v1.fit(data_vb_adj)


train_test_vb_adj = looping_extract_verbs_and_adj(train_test.Phrase)
train_train_vb_adj = looping_extract_verbs_and_adj(train_train.Phrase)
test_vb_adj = looping_extract_verbs_and_adj(test.Phrase)


0
0
0


In [11]:
def add_vb_adj_sentiment(phrases_vb_adj):
    X2 = v1.transform(phrases_vb_adj)

    sentiments = []
    for feature in v1.get_feature_names():
        sentiment = TextBlob(feature).sentiment
        value = sentiment.polarity * sentiment.subjectivity
        sentiments.append(value)
    from scipy.sparse import csr_matrix

    X3 = csr_matrix(X2.shape)
    for index in range(X2.shape[0]):
        row = X2[index]
        for col in range(row.shape[1]):
            X3[index, col] = X2[index, col] * sentiments[col]

    return X3

X_train_test = add_vb_adj_sentiment(train_test_vb_adj)
X_train_train = add_vb_adj_sentiment(train_train_vb_adj)
X_test = add_vb_adj_sentiment(test_vb_adj)

/home/zhecan/anaconda2/lib/python2.7/site-packages/scipy/sparse/compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [12]:

print X_train_train.shape
print X_train_test.shape
print X_test.shape

(5, 9)
(5, 9)
(10, 9)


In [21]:
from sklearn.svm import SVC
clf = SVC()
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

# y = np.array([1,2,3,4,5])
clf.fit(X_train_train, train_train.Sentiment) 

y_train_test_predict = clf.predict(X_train_test)
y_test_predict = clf.predict(X_test)


train_output = pd.DataFrame({
    'PhraseId': train_test.PhraseId,
    'Predicted': y_train_test_predict,
    'Sentiment': train_test.Sentiment
})



test_output = pd.DataFrame({
    'PhraseId': test.PhraseId,
    'Sentiment': y_test_predict
})

5
<class 'pandas.core.series.Series'>
